# Zero coupon curve derivation
This is the Python implementation of the zero coupon curve derivation from swap rates taught in my Capital Markets class at IE Business School. 

Swap rates are scraped from SEB Group at the link below:

https://sebgroup.com/large-corporates-and-institutions/prospectuses-and-downloads/rates/swap-rates


In [1]:
# First, necessary packages are imported. Since the html stores the swap rates in a separte iframe, 
# I use Selenium to access the tables

import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import numpy as np
from decimal import *

getcontext().prec = 4

# Next, I set up the url and options to open Chromedriver in the background with Selenium
url = 'https://sebgroup.com/large-corporates-and-institutions/prospectuses-and-downloads/rates/swap-rates'
CHROMEDRIVER_PATH = '/Users/David/chromedriver'
WINDOW_SIZE = "1920,1080"
options = Options()  
options.add_argument("--headless")  
options.add_argument("--window-size=%s" % WINDOW_SIZE)

# Finally, initialize the webdriver, call the url, store all source html from the appropriate iframe in a list of dataframes 
# using Pandas read_html function, and then close the webdriver
driver = webdriver.Chrome(executable_path=CHROMEDRIVER_PATH,
                          options=options) 
driver.get(url)
driver.switch_to.frame(driver.find_element_by_id('iFrameResizer0'))
dfs = pd.read_html(driver.page_source)
driver.close()    

In [2]:
# The next step is to read all swap price columns into a single dataframe, then divide by 100 to get to percentages
df = pd.DataFrame(index = dfs[1]['Maturity'].str.strip(" Yr").astype('int64'))
print("Pulled swap curve for:\n")
for i in range(len(dfs)-1):
    if i % 2 == 0:
        start = dfs[i][0].values[0].find("[")+1
        end = len(dfs[i][0].values[0])-1
        cur = dfs[i][0].values[0][dfs[i][0].values[0].find("[")+1:end]
        print(cur)
        dfs[i+1].index = dfs[i+1]['Maturity'].str.strip(" Yr").astype('int64')
        df1 = dfs[i+1]['Price']
        df1.name = cur
        df = pd.concat([df, dfs[i+1]['Price']], axis=1, sort=True)

df = df / 100

Pulled swap curve for:

SEK
NOK
DKK
EUR
USD


In [3]:
# Next, I define to functions, the first to calculate the forward curve, which returns the swap curve scraped from SEB, 
# the zero coupon curve, and forward curve

def forward_curve(currency, maturity, df):
    if maturity >10:
        return 
    # Create DataFrame of correct size, initialized with zeros
    df1 = pd.DataFrame(np.zeros((maturity,maturity)), index = range(1,maturity+1), columns =range(1,maturity+1))
    
    # Extract the swap curve for the approprate maturity from the original dataframe
    swapc = df[currency][:maturity].values
    
    # Loop through each time period for the maturity selected and create series of cash flows
    for i in range(0,maturity):
        index = i+1
        # Replicate swap rate for all periods up until maturity
        l = [df[currency][index]]*(df[currency].index[i])
        # Adjust final cash flow to include principal repayment
        l[-1] = l[-1] + 1
        # Write data to DataFrame
        df1.iloc[i,:index] = l
    # Calculate inverse matrix
    inv = np.linalg.inv(df1.iloc[0:maturity, 0:maturity])
    # Create array of discount periods
    disc_per = np.array(range(1,maturity+1))
    zcc = 1 / np.dot(inv,np.ones(maturity))**(1/disc_per)-1
    fc = ( 1+ zcc[1:])**disc_per[1:] / ((1+zcc[:-1])**disc_per[:-1])-1
    fc = np.insert(fc, 0, zcc[0])
    return swapc, zcc, fc

# This function calls the forward_curve function, then combines all the curves and both nominal and present value of fixed bonds and floater bonds
def fixed_vs_float(currency, maturity, df):
    if maturity >10:
        return 
    swapc, zcc, fc = forward_curve(currency, maturity, df)
    fixed = np.array([df[currency][maturity]] * maturity)
    fixed[maturity-1] = fixed[maturity-1] + 1
    disc_per = np.array(range(1,maturity+1))
    pv_fixed = fixed / (1+ zcc)**disc_per
    floater = fc
    floater[maturity-1] = 1 + floater[maturity-1]
    pv_floater = floater / (1+zcc)**disc_per
    full_df = pd.DataFrame(data = [swapc, zcc, fc, fixed, pv_fixed, floater, pv_floater]).T
    full_df.columns = ['Swap Curve', 'Zero Coupon Curve', 'Forward Curve', 'Fixed Coupons', 'PV Fixed', 'Floater', 'PV Floater']
    full_df.index = disc_per
    full_df.index.rename('Maturity', inplace=True)
    return full_df


In [4]:
# Test the function for a 5 year USD note
curves = fixed_vs_float("USD", 10, df)

# Check that PV of both notes equals 100%
print("Floater value: ", round(sum(curves['PV Floater']),2))
print("Fixed note value: ", round(sum(curves['PV Fixed']), 2))

# Display entire curve dataframe
curves

Floater value:  1.0
Fixed note value:  1.0


,Swap Curve,Zero Coupon Curve,Forward Curve,Fixed Coupons,PV Fixed,Floater,PV Floater
Maturity,,,,,,,
1,0.0200,0.020000,0.020000,0.0196,0.019216,0.020000,0.019608
2,0.0178,0.017780,0.015566,0.0196,0.018921,0.015566,0.015027
3,0.0173,0.017279,0.016276,0.0196,0.018618,0.016276,0.015460
4,0.0173,0.017284,0.017300,0.0196,0.018302,0.017300,0.016154
5,0.0175,0.017494,0.018335,0.0196,0.017972,0.018335,0.016812
6,0.0179,0.017913,0.020012,0.0196,0.017619,0.020012,0.017990
7,0.0183,0.018334,0.020864,0.0196,0.017259,0.020864,0.018373
8,0.0187,0.018758,0.021729,0.0196,0.016892,0.021729,0.018727
9,0.0192,0.019294,0.023587,0.0196,0.016503,0.023587,0.019860
